# Cvičenie 3: Bellmanove rovnice

Na minulom cvičení sme zadefinovali jednoduché prostredie gridworld ako Markovov rozhodovací proces. Na dnešnom cvičení sa pozrieme na to, ako vieme vyriešiť problém, ktorý predstavuje toto prostredie - teda nájsť optimálnu politiku na pohybovanie sa v gridworlde tak, aby sme sa dostali na cieľovú pozíciu a zároveň aby sme sa vyhli pasci. Využijeme pritom Bellmanove rovnice očakávania a funkciu optimality.

## Gridworld

Výpočty budeme aplikovať pre konkrétny gridworld veľkosti *3x3* s jedným cieľom a jednou pascou. Gridworld je plne deterministický, teda pravdepodobnosť prechodov je buď 0 alebo 1. Cieľová pozícia a pasca sú terminálne stavy, takže vieme, že ich očakávaná hodnota je 0.

<img src="sources/lab03/gridworld_example2.jpg" width="300">

Daný svet vieme reprezontovať formálne ako MDP. Pri prechodoch sú označené akcie a odmena pre vykonanie danej akcie (pravdepobnosť označených prechodov je 1). Pre jednoduchosť opravíme aj označenie stavov.

<img src="sources/lab03/gridworld_mdp.jpg" width="600">

## Bellmanova rovnica očakávania

V prvom kroku vypočítame očakávanú hodnotu jednotlivých stavov pri náhodnej politike. Použijeme na to Bellmanovu rovnicu očakávania:

$v_{\pi}(s) = \sum_{a \in \mathcal{A}} \pi(a|s) \ (r(s, a) + \gamma \sum_{s' \in \mathcal{S}} p(s'|s, a) \  v_{\pi}(s'))$

Keďže politika je náhodná, $\pi(a|s) = 0.25$ pre všetky stavy. Diskontný faktor $\gamma = 0.8$.

Napríklad pre stav $s_0$ vieme vypočítať hodnotu nasledovne:

$v_{\pi}(s_0) = \pi(N|s_0) \cdot (r(s_0, N) + \gamma \cdot p(s_0|s_0, a) \cdot v_{\pi}(s_0)) + \\ + \pi(E|s_0) \cdot (r(s_0, E) + \gamma \cdot p(s_1|s_0, a) \cdot v_{\pi}(s_1)) + \\ + \pi(S|s_0) \cdot (r(s_0, S) + \gamma \cdot p(s_3|s_0, a) \cdot v_{\pi}(s_3)) + \\ + \pi(W|s_0) \cdot (r(s_0, W) + \gamma \cdot p(s_0|s_0, a) \cdot v_{\pi}(s_0))$

$v_{\pi}(s_0) = 0.25 \cdot (-1.0 + 0.8 \cdot v_{\pi}(s_0)) + 0.25 \cdot (-1.0 + 0.8 \cdot v_{\pi}(s_1)) + 0.25 \cdot (-10.0 + 0.8 \cdot 0.0) + 0.25 \cdot (-1.0 + 0.8 \cdot v_{\pi}(s_0))$

$v_{\pi}(s_0) = -0.25 + 0.2 \cdot v_{\pi}(s_0) - 0.25 + 0.2 \cdot v_{\pi}(s_1) -2.5 + 0.0 -0.25 + 0.2 \cdot v_{\pi}(s_0)$

$v_{\pi}(s_0) = -3.25 + 0.4 \cdot v_{\pi}(s_0) + 0.2 \cdot v_{\pi}(s_1)$

Keď to upravíme na formu lineárnych rovníc:

$0.6 \cdot v_{\pi}(s_0) - 0.2 \cdot v_{\pi}(s_1) = -3.25$

**Úloha:** Zostavte rovnice pre ostatné stavy (okrem $s_2$ a $s_3$).

Pre všetky stavy dostaneme spolu sedem lineárnych rovníc o sedem neznámych. Následne môžeme vyriešiť túto sústavu a tak dostaneme očakávané hodnoty $v_{\pi}(s)$ pri politike $\pi$.

**Úloha:** Vyriešte sústavu zostavených lineárnych rovníc v Pythone. Použite pritom funkciu `numpy.linalg.solve`.

In [ ]:
import numpy as np

A = np.array([
    [],
    [],
    [],
    [],
    [],
    [],
    []
])
B = np.array([
    
])
X = np.linalg.solve(A, B)

## Bellmanova funkcia optimality

Bellmanove rovnice môžeme využiť aj pri hľadaní optimálnej politiky pre daný svet. Zameriame sa pritom na očakávanú hodnotu dvojíc stav-akcia, teda chceme zistiť, ako veľmi je daná akcia vhodná pre daný stav. Hodnotu určíme podľa vzorca:

$q(s, a) = r(s, a) + \gamma \sum_{s' \in \mathcal{S}} p(s'|s, a) \cdot v(s')$

Akcia je optimálna ak maximalizuje odmeny počas interakcie:


$q_{*}(s, a) = r(s, a) + \gamma \sum_{s' \in \mathcal{S}} p(s'|s, a) \cdot v_{*}(s')$

$q_{*}(s, a) = \sum_{r \in \mathcal{R}} \sum_{s' \in \mathcal{S}} p(s', r|s, a) (r + \gamma \cdot \underset{a}{max} q_{*}(s', a'))$

Pre náš príklad teda vieme napísať rovnice (považujeme iba jeden prechod keďže svet je deterministický):

$q(s_0, N) = p(s_0, r(s_0, N)|s, N) \cdot (r(s_0, N) + \gamma \cdot v_{*}(s_0))$

$q(s_0, N) = 1 \cdot (-1 + 0.8 \cdot v_{*}(s_0))$

$q(s_0, N) = -1 + 0.8 \cdot v_{*}(s_0)$

$q(s_0, N) - 0.8 \cdot v_{*}(s_0) = -1$

A obdobne pre ďalšie akcie:

$q(s_0, E) - 0.8 \cdot v_{*}(s_1) = -1$

$q(s_0, S) - 0.8 \cdot v_{*}(s_3) = -10 \rightarrow q(s_0, S) = -10$

$q(s_0, W) - 0.8 \cdot v_{*}(s_0) = -1$

**Úloha:** Napíšte rovnice pre všetky zvyšné kombinácie stav-akcia (okrem stavov $s_2$ a $s_3$). Vychádzame zo znalosti, že $v_{*}(s_2) = 0$ a $v_{*}(s_3) = 0$.

Dostali sme 28 rovníc (7 stavov $\times$ 4 akcie), máme ale spolu 35 neznámych (28 hodnôt stav-akcia + 7 maximálnych hodnôt stavov). Našu sústavu môžeme rozšíriť o predpoklady, že maximálna hodnota stavu musí byť najväčšie z hodnôt stav-akcia pre daný stav, napr.:

$v_{*}(s_0) = max(q(s_0, N), q(s_0, E), q(s_0, S), q(s_0, W))$

To ale ešte stále nevieme jednoducho napísať ako rovnicu, práve preto hodnoty vypočítame pomocou SMT solvera. **Úloha:** Zapíšte rovnice a predpoklady do SMT solvera Z3. [Môžete použiť online nástroj.](https://compsys-tools.ens-lyon.fr/z3/index.php) [Vaše riešenie môže vyzerať takto.](sources/lab03/gridworld_smt.smt)

Alternatívne riešenie je vyskúšať všetky možné politiky a zistiť, či sú optimálne. Takto naše predpoklady na $v_{*}(s)$ vieme zapísať ako rovnicu, napríklad ak akcia $E$ je optimálna zo stavu $s_0$: $v_{*}(s_0) - q(s_0, E) = 0$. Následne vyriešime sústavu lineárnych rovníc a skontrolujeme si, či hodnota $v_{*}(s)$ je naozaj najväčšia z hodnôt $q(s, a) \forall a \in \mathcal{A}$. [Ukážkové riešenie v Pythone nájdete tu.](sources/lab03/bellman_gridworld.py)